<a href="https://colab.research.google.com/github/MatteoAldovardi92/Datascience-and-Machine-Learning-Sandbox/blob/main/MiniGpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
# Install the datasets library
!pip install datasets==2.19.0 fsspec==2024.3.1 aiohttp==3.9.3


In [79]:
from datasets import load_dataset

# Load the WikiText-103 dataset
dataset = load_dataset("wikitext", "wikitext-103-v1")

# You can explore the dataset structure
print(dataset)

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 1801350
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})


In [80]:
#dataset['train']['text']

In [81]:
!wget "https://raw.githubusercontent.com/karphathy/char-rnn/blob/master/data/tinyshakespeare/input.txt"

--2025-06-27 15:16:46--  https://raw.githubusercontent.com/karphathy/char-rnn/blob/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-06-27 15:16:46 ERROR 404: Not Found.



In [82]:
# Attempt to download the raw tinyshakespeare input.txt file
!wget "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"

--2025-06-27 15:16:46--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.4’

input.txt.4         100%[===================>]   1.06M  --.-KB/s    in 0.009s  

2025-06-27 15:16:46 (116 MB/s) - ‘input.txt.4’ saved [1115394/1115394]



In [83]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [84]:
characters = sorted(list(set(text))) #naaaah maaan! naaah!

characters

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [85]:
vocab_size = len(characters)


In [86]:
stoi = { ch:i for i,ch in enumerate(characters) }
itos = { i:ch for i,ch in enumerate(characters)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [87]:
import tiktoken
enc = tiktoken.get_encoding("o200k_base")
assert enc.decode(enc.encode("hello world")) == "hello world"

# To get the tokeniser corresponding to a specific model in the OpenAI API:
enc = tiktoken.encoding_for_model("gpt-4o")

# Use the encode method to encode the string
encoded_text = enc.encode("hii there!")
print(encoded_text)

# Use the decode method to decode the token ids back to a string
decoded_text = enc.decode(encoded_text)
print(decoded_text)


# These encoders trade length of the encoded string with the length
# of the vocabulary but this lesson we won't use them



[71, 3573, 1354, 0]
hii there!


In [88]:
import torch
tiny_shakespeare = torch.tensor(encode(text), dtype=torch.long)
print(tiny_shakespeare.shape, tiny_shakespeare.dtype)
print(tiny_shakespeare[:10])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


In [89]:
n = int(0.9*len(tiny_shakespeare))
train_data = tiny_shakespeare[:n]
val_data = tiny_shakespeare[n:]


In [90]:
block_size = 8 #context size
train_data[:block_size+1]



tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

**When training a transformer, for each block, we are essentially feeding the model a number of examples equal to the `block_size`.**

In [91]:

examples = {}

for i in range(block_size):
    context = train_data[:i+1].tolist()  # Convert tensor to list
    target = train_data[i+1].item()    # Convert scalar tensor to scalar
    examples[f"example {i + 1}: (context, target)"] = (context, target)

examples

{'example 1: (context, target)': ([18], 47),
 'example 2: (context, target)': ([18, 47], 56),
 'example 3: (context, target)': ([18, 47, 56], 57),
 'example 4: (context, target)': ([18, 47, 56, 57], 58),
 'example 5: (context, target)': ([18, 47, 56, 57, 58], 1),
 'example 6: (context, target)': ([18, 47, 56, 57, 58, 1], 15),
 'example 7: (context, target)': ([18, 47, 56, 57, 58, 1, 15], 47),
 'example 8: (context, target)': ([18, 47, 56, 57, 58, 1, 15, 47], 58)}

### Create batches

In [92]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def create_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y



xb, yb = create_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')


for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [93]:
from logging import logMultiprocessing
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
  def forward(self, idx, targets = None):
    if targets == None:
      loss = None
      logits = self.token_embedding_table(idx) # (B, T, C)
      return logits, loss
    else:
      logits = self.token_embedding_table(idx) # (B, T, C)
      loss = F.cross_entropy(logits.view(-1, logits.shape[-1]), targets.view(-1)) #(B*T, C),
      return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, loss = self(idx, None)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1)
    return idx


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)


torch.Size([4, 8, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


# Optimize the loss

In [94]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [95]:
batch_size = 32
n_steps = 10000
for steps in range(n_steps):
  xb, yb = create_batch('train')
  logits, loss = m(xb, yb)
  loss.backward()
  optimizer.step()

print(loss.item())

2.766160726547241


In [96]:
print(decode(m.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


I oteng h hasbe pave pirance
RDe hicomyonthar's
PES:
ARod ith heno hatucenonthioneir thondy, y helti
